## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию.

Считываю таблицу и вывожу первые 10 строк, чтобы посмотреть данные и названия столбцов.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывожу общую информацию о таблице, чтобы убедиться, что тип данных правильный в каждом стобце.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Смотрю, какие значения встречаются в каждом столбце, чтобы проверить на наличие артефактов.

In [3]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [4]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [5]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [6]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [7]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [8]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [9]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [10]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [11]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [12]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Названия столбцов данной таблицы являются приемлемыми для дальнейшей работы. В некоторый стоблцах имеются пропущенные и ошибочные значения. Имеется разный регистр в столбце. Один стобец имеет неправильный тип данных. В дальших шагах все эти ошибки будут обработаны.

### Шаг 2. Предобработка данных

### Обработка пропусков

Пропущенные значения имеются в  столбцах days_employed и total_income, а некорректные - только в первом. Оценим, количество пропусков и некорректных данных:

In [13]:
count_incorrect = data[data['days_employed'] < 0]['days_employed'].count() # общее количество некорректных данных
total = data.shape[0] # количество всех строк в таблице
total_null = data[data['days_employed'].isnull()].count()[0] # количество пропущенных данных
total_correct = total - data[data['days_employed'].isnull()].count()[0] #количество непустых значений в таблице

Доля пропущенных данных:

In [14]:
(total_null) / total

0.10099883855981417

Доля некорректных данных:

In [15]:
count_incorrect / total_correct

0.8219730246498889

Доля пропущенных значений составляет 10% процентов, в дальнейшем этот столбец использоваться не будет, но и удаление строк с пустыми данными неприемлемо, так как потеряется часть полезных данных, поэтому можно заполнить пропуски 0.
А вот доля некорректных данных гораздо больше, но делать с ними ничего не будем.

In [16]:
data = data.fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Так же некорректные данные есть в столбце children. Таких значений очень мало, поэтому я принял решение удалить эти строчки.

In [17]:
data = data[data['children'] >= 0]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
children            21478 non-null int64
days_employed       21478 non-null float64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null float64
purpose             21478 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Пропущенные значения в столбцах были устранены, посредством замены пропуска на 0. В условиях данной задачи это является корректным решением.

### Замена типа данных

В столбцах days_employed и total_income заменим тип данных с float на int.

In [18]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
children            21478 non-null int64
days_employed       21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null int64
purpose             21478 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Была успешная произведена замена типов данных с float на int в столбцах days_employed и total_income.

### Обработка дубликатов

В данной таблице нет никаких уникальных идентификаторов, как например id клиента. Поэтому дубликатами буду считать строки, которые полностью идентичны. Но перед этим сначала устраняю проблему с разным регистром в столбце education.

In [19]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15196
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Нахожу количество дубликатов и удаляю их:

In [20]:
data.duplicated().sum()

71

In [21]:
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21407 entries, 0 to 21524
Data columns (total 12 columns):
children            21407 non-null int64
days_employed       21407 non-null int64
dob_years           21407 non-null int64
education           21407 non-null object
education_id        21407 non-null int64
family_status       21407 non-null object
family_status_id    21407 non-null int64
gender              21407 non-null object
income_type         21407 non-null object
debt                21407 non-null int64
total_income        21407 non-null int64
purpose             21407 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Мною были успешно удалены дубликаты в таблице.

### Лемматизация

Чтобы разделить все цели кредита на категории, необходимо применить лемматезацию к столбцу purpose. Для еще раз посмотрим на данные в этом столбце, чтобы выделить примерные категории.

In [22]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           764
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
операции с жильем                         651
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     649
жилье                                     645
покупка жилья                             643
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              632
операции со своей недвижимостью           624
строительство жилой недвижимости          621
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              606
покупка жилой недвижимости                604
на покупку своего автомобиля              504
заняться высшим образованием      

Я выделяю 4 категории (5-я категория не используется, служит для подстраховки) и создаю функцию для сортировки:

In [23]:
def categories(row):
    if 'образование' in row:
        return 'образование'
    elif 'автомобиль' in row:
        return 'авто'
    elif ('недвижимость' in row) or ('жилье' in row):
        return 'недвижимость'
    elif 'свадьба' in row:
        return 'свадьба'
    else:
        return 'разное'

Одновременно использую лемматизацию и применяю функцию сортировки.

In [24]:
from pymystem3 import Mystem
m = Mystem()
data['categories'] = data['purpose'].apply(m.lemmatize).apply(categories)

Результат:

In [25]:
data['categories'].value_counts()

недвижимость    10787
авто             4295
образование      4003
свадьба          2322
Name: categories, dtype: int64

In [26]:
data.loc[:, ['purpose', 'categories']].head(10)

,purpose,categories
0,покупка жилья,недвижимость
1,приобретение автомобиля,авто
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
5,покупка жилья,недвижимость
6,операции с жильем,недвижимость
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,недвижимость


### Вывод

Все цели были успешно разделены на категории.

### Категоризация данных

Для ответа на вопросы проекта, я составлю следующие словари:

1. Сумма дохода - статус дохода.

Доходы выше медианного значения - высокий доход, меньше - низкий.

In [27]:
median_income = data.sort_values('total_income')['total_income'].median() #Нахожу медиану
median_income

135760.0

Функция для сортировки:

In [28]:
def total_income_status(row):
    if row > median_income:
        return 'высокий'
    else:
        return 'низкий'

Применяю функцию и вывожу словарь:

In [29]:
data['total_income_status'] = data['total_income'].apply(total_income_status)

In [30]:
data.loc[:, ['total_income', 'total_income_status']].head(10)

,total_income,total_income_status
0,253875,высокий
1,112080,низкий
2,145885,высокий
3,267628,высокий
4,158616,высокий
5,255763,высокий
6,240525,высокий
7,135823,высокий
8,95856,низкий
9,144425,высокий


2. Тип дохода - статус дохода.

Этот словарь уменьшает количество типов дохода до 3 для более удобной обработки.

Функция для сортировки:

In [31]:
def income_status(row):
    if row == 'сотрудник' or row == 'госслужащий':
        return 'работник'
    elif row == 'компаньон' or row == 'предприниматель':
        return 'бизнесмен'
    else:
        return 'безработный'

Применяю функцию и вывожу словарь:

In [32]:
data['income_status'] = data['income_type'].apply(income_status)
data.loc[:, ['income_type', 'income_status']].head(10)

,income_type,income_status
0,сотрудник,работник
1,сотрудник,работник
2,сотрудник,работник
3,сотрудник,работник
4,пенсионер,безработный
5,компаньон,бизнесмен
6,компаньон,бизнесмен
7,сотрудник,работник
8,сотрудник,работник
9,сотрудник,работник


3. Количество детей - статус количества детей.

Этот словарь разделяет количество детей на 3 группы.

Функция для сортировки:

In [33]:
def children_count(row):
    if row == 0:
        return 'без_детей'
    elif row == 1:
        return '1_ребенок'
    elif row > 1:
        return 'многодетный'

Применяю функцию и вывожу словарь:

In [34]:
data['children_status'] = data['children'].apply(children_count)
data.loc[:, ['children', 'children_status']].head(10)

,children,children_status
0,1,1_ребенок
1,1,1_ребенок
2,0,без_детей
3,3,многодетный
4,0,без_детей
5,0,без_детей
6,0,без_детей
7,0,без_детей
8,2,многодетный
9,0,без_детей


4. Возврат кредита в срок - возврат кредита в срок.

Этот словарь соотносит числовой статус в статус в виде строки (для удобства создания таблиц).

Функция для сортировки:

In [35]:
def statusoplati(row):
    if row == 0:
        return 'надежный'
    if row == 1:
        return 'ненадежный'

Применяю функцию и вывожу словарь:

In [36]:
data['debt_status'] = data['debt'].apply(statusoplati)
data.loc[:, ['debt', 'debt_status']].head(15)

,debt,debt_status
0,0,надежный
1,0,надежный
2,0,надежный
3,0,надежный
4,0,надежный
5,0,надежный
6,0,надежный
7,0,надежный
8,0,надежный
9,0,надежный


5. Количество лет - возрастная группа.

Распределение людей по возрастным группам.

Функция для сортировки:

In [37]:
def age_status(row):
    if row <=27:
        return 'молодой'
    elif 27 < row < 60:
        return 'взрослый'
    elif row > 60:
        return 'пожилой'

Применяю функцию и вывожу словарь:

In [38]:
data['age_status'] = data['dob_years'].apply(age_status)
data.loc[:, ['dob_years', 'age_status']].head(10)

,dob_years,age_status
0,42,взрослый
1,36,взрослый
2,33,взрослый
3,32,взрослый
4,53,взрослый
5,27,молодой
6,43,взрослый
7,50,взрослый
8,35,взрослый
9,41,взрослый


### Вывод

Мною были созданы 4 словаря для дальнейшего их использования в поиске закономерностей.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос сделаю сводную таблицу.

In [39]:
grouped_by_children = data.pivot_table(index=['children_status'], columns='debt_status', values='debt', aggfunc='count')

In [40]:
grouped_by_children['percent_of_debt'] = grouped_by_children['ненадежный'] / (grouped_by_children['надежный'] + grouped_by_children['ненадежный']) * 100
grouped_by_children.sort_values('percent_of_debt')

debt_status,надежный,ненадежный,percent_of_debt
children_status,,,
без_детей,13028,1063,7.543822
1_ребенок,4364,444,9.234609
многодетный,2275,233,9.290271


### Вывод

Видно, что люди без детей чаще возвращают кредиты в срок. И чем меньше количество детей, тем выше шанс, что оплата будет вовремя произведена.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Также сделаем сводную таблицу.

In [41]:
grouped_by_family = data.pivot_table(index=['family_status'], columns='debt_status', values='debt', aggfunc='count')

In [42]:
grouped_by_family['percent_of_debt'] = grouped_by_family['ненадежный'] / (grouped_by_family['надежный'] + grouped_by_family['ненадежный']) * 100
grouped_by_family.sort_values('percent_of_debt')

debt_status,надежный,ненадежный,percent_of_debt
family_status,,,
вдовец / вдова,892,63,6.596859
в разводе,1106,85,7.136860
женат / замужем,11380,930,7.554833
гражданский брак,3758,388,9.358418
Не женат / не замужем,2531,274,9.768271


И еще одну.

In [43]:
grouped_by_family_and_age = data.pivot_table(index=['family_status', 'age_status'], columns='debt_status', values='debt', aggfunc='count')

In [44]:
grouped_by_family_and_age['percent_of_debt'] = grouped_by_family_and_age['ненадежный'] / (grouped_by_family_and_age['надежный'] + grouped_by_family_and_age['ненадежный']) * 100
grouped_by_family_and_age.sort_values('percent_of_debt')

debt_status                       надежный  ненадежный  percent_of_debt
family_status         age_status                                       
Не женат / не замужем пожилой          188           7         3.589744
гражданский брак      пожилой          322          14         4.166667
женат / замужем       пожилой         1063          50         4.492363
вдовец / вдова        пожилой          329          21         6.000000
                      взрослый         512          35         6.398537
в разводе             взрослый         905          66         6.797116
                      пожилой          119           9         7.031250
женат / замужем       взрослый        9299         774         7.683907
гражданский брак      молодой          452          47         9.418838
                      взрослый        2930         323         9.929296
Не женат / не замужем взрослый        1652         184        10.021786
женат / замужем       молодой          832          98        10.537634
Не женат / не замужем молодой          649          81        11.095890
в разводе             молодой           57           8        12.307692
вдовец / вдова        молодой            6           1        14.285714

### Вывод

Лучше всего отдают кредиты в срок вдовцы и вдвовы в пожилом возрасте. Но данных для этого случая немного, поэтому использовать это не буду. Далее идут люди пожилого и среднего возраста которые были или есть в отношениях. Это хороший показатель. Хуже всего отдают молодые люди с и без отношений. Из этого можно сделать вывод, что возраст играет большее значение, чем семейное положение.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сделаем сводную таблицу.

In [45]:
grouped_by_finance = data.pivot_table(index=['total_income_status',], columns='debt_status', values='debt', aggfunc='count')

In [46]:
grouped_by_finance['percent_of_debt'] = grouped_by_finance['ненадежный'] / (grouped_by_finance['надежный'] + grouped_by_finance['ненадежный']) * 100
grouped_by_finance.sort_values('percent_of_debt')

debt_status,надежный,ненадежный,percent_of_debt
total_income_status,,,
высокий,9839,864,8.072503
низкий,9828,876,8.183857


И еще одну.

In [47]:
grouped_by_income_type = data.pivot_table(index=['income_status'], columns='debt_status', values='debt', aggfunc='count')

In [48]:
grouped_by_income_type['percent_of_debt'] = grouped_by_income_type['ненадежный'] / (grouped_by_income_type['надежный'] + grouped_by_income_type['ненадежный']) * 100
grouped_by_income_type.sort_values('percent_of_debt')

debt_status,надежный,ненадежный,percent_of_debt
income_status,,,
безработный,3607,218,5.699346
бизнесмен,4695,376,7.414711
работник,11365,1146,9.159939


### Вывод

Связи между уровнем дохода и возвратом кредита в срок нету. А вот от типа дохода есть. Безработные (большая часть это пенсионеры) лучше всего отдают кредиты. Хуже всего обычные рабочие. Бизнесмены занимают промежуточное значение.

- Как разные цели кредита влияют на его возврат в срок?

Также сделаем сводную таблицу.

In [49]:
grouped_by_purpose = data.pivot_table(index=['categories'], columns='debt_status', values='debt', aggfunc='count')

In [50]:
grouped_by_purpose['percent_of_debt'] = grouped_by_purpose['ненадежный'] / (grouped_by_purpose['надежный'] + grouped_by_purpose['ненадежный']) * 100
grouped_by_purpose.sort_values('percent_of_debt')

debt_status,надежный,ненадежный,percent_of_debt
categories,,,
недвижимость,10005,782,7.249467
свадьба,2136,186,8.010336
образование,3633,370,9.243068
авто,3893,402,9.359721


### Вывод

Кредиты, связанные с недвижимостью, чаще всех возвращаются вовремя. Вероятно это связано с тем, что недвижимость - хорошее финансовая инвестиция. А вот кредиты на авто хуже всего, может быть это связано с большим количеством мошенничества в этой сфере.

### Шаг 4. Общий вывод

В данном проекте я искал зависимости между разными данными клиентов банков и возвратом кредита в срок. Для этого сначала я считал данные, обработал пропущенные значения, убрал дубликаты, категоризовал данные и построил несколько сводных таблиц.В итоге, зависимости были обнаружены у целей кредита, возрастом, занимаемой должностью и количеством детей.